# TV Script Generation

In this project,the goal is to generate new fake  [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network  will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data can be found here [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) 

In [2]:
from collections import Counter

In [3]:
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Playing around with `view_line_range` to view different parts of the data. This will help to get  a sense of the data we are working with.For example, it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [4]:
view_line_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.

### Lookup Table
To create a word embedding, we first need to transform the words to ids.  In this function,  I created two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

In [5]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # create a dict for words along with their respective counts and then sort from the least frequent to the most frequent
    words = Counter(text)
    words = sorted(words , key = words.get, reverse=True)
    # create a dict vocab_to_int
    vocab_to_int = {word: index for index, word in enumerate(words)}
    #create a dict int_to_vocab
    int_to_vocab = {index : word for index , word in enumerate(words)}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

the function `token_lookup`  returns a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # create a dict for tokens 
    result = dict()
    result['.'] = '<PERIOD>'
    result[','] = '<COMMA>'
    result['"'] = '<QUOTATION_MARK>'
    result[';'] = '<SEMICOLON>'
    result['!'] = '<EXCLAMATION_MARK>'
    result['?'] = '<QUESTION_MARK>'
    result['('] = '<LEFT_PAREN>'
    result[')'] = '<RIGHT_PAREN>'
    result['?'] = '<QUESTION_MARK>'
    result['-'] = '<DASH>'
    result['\n'] = '<NEW_LINE>'
        
    return result

tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file.

In [7]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point

In [8]:
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, I built the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [10]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implementing the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

In [ ]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # get the number of batches 
    nbr_batches = len(words) // batch_size
    # keep only full batches 
    words = words[:nbr_batches * batch_size]
    x, y = [], []
    limit = len(words) - sequence_length
    for i in range(0, limit):
        end = sequence_length + i
        x_batch = words[i:end]
        x.append(x_batch)
        batch_y =  words[end]  
        y.append(batch_y) 
    
    # create a Tensor 
    data = TensorDataset(torch.from_numpy(np.asarray(x)), torch.from_numpy(np.asarray(y)))
    # create a data loader 
    data_loader = DataLoader(data, shuffle=False, batch_size=batch_size)

    return data_loader   
   

### Test the dataloader 

Expected return :<br>
```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

Notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [12]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0,   1,   2,   3,   4],
        [  1,   2,   3,   4,   5],
        [  2,   3,   4,   5,   6],
        [  3,   4,   5,   6,   7],
        [  4,   5,   6,   7,   8],
        [  5,   6,   7,   8,   9],
        [  6,   7,   8,   9,  10],
        [  7,   8,   9,  10,  11],
        [  8,   9,  10,  11,  12],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14])


---
## Build the Neural Network
Implementing an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module).A GRU or an LSTM can serve in this usecase as well.
 
**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.


In [13]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM layers
        """
        super(RNN, self).__init__()
        
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # set the model's layers 
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        
    
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """ 
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_output, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_output  = lstm_output.contiguous().view(-1, self.hidden_dim)
        
        
        #  fully-connected layer
        out = self.fc(lstm_output)
        
        # reshape into (batch_size, seq_length, output_size) and get the last batch 
        out = out.view(batch_size, -1, self.output_size)[:, -1]
        
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if(train_on_gpu):
            result = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(), 
                    weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda() )         
        else:
            result = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(), 
                    weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()) 
        
        return result


tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

In [14]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    
    # move data & the model  to GPU, if available
    if(train_on_gpu):
        rnn.cuda()
        inp = inp.cuda()
        target = target.cuda()
    
    # perform backpropagation and optimization
    # 1- get the hidden state 
    hidden_state = tuple([each.data for each in hidden])
    
    # 2- get rid of the cumulative grad
    rnn.zero_grad()
    
    #3 - get the output = forward pass
    output, hidden_state = rnn(inp, hidden_state)
    
    # 4 - calculate the  loss 
    loss = criterion(output, target)
    
    # 5- Do a backward pass
    loss.backward()
    
    #6- clip grad norm to prevent the exploding of the gradient values 
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    
    # 7- update the weights 
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden_state

tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

In [15]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [16]:
# Data params
# Sequence Length
sequence_length = 11  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [17]:
# Training parameters
# Number of Epochs
num_epochs = 20 # started with 6
# Learning Rate
learning_rate = 0.001 # started with 0.01 

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 209 # this to ensure that embedding_dim < vocab_size 
# Hidden Dimension
hidden_dim = 245
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500

### Train


In [18]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained 
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')lj

Training for 20 epoch(s)...
Epoch:    1/20    Loss: 5.8985658826828

Epoch:    1/20    Loss: 5.539129868030548

Epoch:    1/20    Loss: 4.931050651550293

Epoch:    1/20    Loss: 4.758363451004028

Epoch:    1/20    Loss: 4.729002011299134

Epoch:    1/20    Loss: 4.747441439628601

Epoch:    1/20    Loss: 4.6413246760368345

Epoch:    1/20    Loss: 4.508276138782501

Epoch:    1/20    Loss: 4.465846632003784

Epoch:    1/20    Loss: 4.409191075801849

Epoch:    1/20    Loss: 4.50649828004837

Epoch:    1/20    Loss: 4.52492133140564

Epoch:    1/20    Loss: 4.520627060413361

Epoch:    2/20    Loss: 4.333784115954864

Epoch:    2/20    Loss: 4.193955753326416

Epoch:    2/20    Loss: 4.076763752937317

Epoch:    2/20    Loss: 4.040309221744537

Epoch:    2/20    Loss: 4.0877908806800844

Epoch:    2/20    Loss: 4.158573823928833

Epoch:    2/20    Loss: 4.087332933902741

Epoch:    2/20    Loss: 3.9855303282737733

Epoch:    2/20    Loss: 3.9686239380836486

Epoch:    2/20    Loss: 3.

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Reflection : How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** <br> 
-  Model's hyperparameters :
    * I have chosen the number of layers to be 3 based on this paper <a href = "https://wiki.inf.ed.ac.uk/twiki/pub/CSTR/ListenTerm1201415/sak2.pdf"> Long Short-Term Memory Recurrent Neural Network Architectures for Large Scale Acoustic Modeling</a> that suggests that a number of layers between 2 to 5 can be efficient for a large number of tasks while something that exceeds 7 layers is computationally expensive in trainning.
    * hidden dimension : I set the hidden dimension to be 245 after I tried multiple values it seemed to be a good value to help the model detect some in depth patterns .
    * Sequence length : I set it to 11 because 11 words are enough to get a meaningfull sentence in a script .
- Optimizer's hyperparameters : 
    * learning rate : I first started with a learning rate of 0.01 and trained for about 7 epochs then I noticed that my loss decreased dramatically for in the two first epochs then tended to be stuck around 5.2  for the rest of epochs , so my model was overshooting the minimum because the learning rate was too large , so I tried in order : 0.009 , 0.006 , 0.003 and finally 0.001 with 20 epochs which resulted in the best performance so far as the overall trend of the network was decreasing and I believe with more epochs we can reach even a lower loss value . 
    * epochs : generally I start with a small number of epochs like 5 or 6 and observe the loss behaviour so if my loss keeps swaying up and down around the same value this means my model is overshooting the minimum and it's better to change the hyperparameters , once I  get an overall decreasing trend over 6 epochs , I increase the number of epochs to 10 then 16 and 20 or more .
    * Mini-batch size : in the hyper parameters lesson they suggest that a good size for minibatch is in a range from 32 to 256 , I first started with a size of 32 and my model was converging slowly then I decided to increase it step by step to 64 and then 128 which resulted in a good performance ( a good convergence rate ) .

# Checkpoint

In [19]:
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
Use the trained network to generate a new, "fake" Seinfeld TV script.

### Generate Text

In [20]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"


In [40]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'elaine' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:42: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


elaine: beep, tear out to the table and the stand.

kramer: well, i am a cheater.

elaine: what?

newman: well it's a good one of the incident.

elaine: oh. i don't know, i was wondering that you want to do it anymore.

kramer: yeah, and you shoulda seen a lot of people, and i don't have to get a little attorney.

jerry:(leaving) no, no, no. no. no, no.

morty: so you're not going to get the victim, and i can't get this out of the way.

jerry: i don't understand.

kramer:(bitter) well, i don't want to get a picture of it.

jerry: well, i'm gonna go anywhere, jerry.

george: i don't care.

elaine:(pointing) no, no, no.

jerry:(shocked) you want to see me?

george: no, no.

jerry:(to the phone) yeah, i got the phone from my life, and the water was the hunter.

george: what?

kramer: oh, i think i'm comin' up with my parents to protect the catalog.

jerry:(laughing) oh, yeah..

george: so?

jerry: i know.

jerry: so, what did you do?

jerry: i'm gonna go anywhere.

jerry: i don't know, bu

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [41]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 